In [2]:
import sys
import os

In [3]:
os.environ.get('JAVA_HOME')

'C:\\Program Files\\Java\\jdk1.8.0_311'

In [4]:
import findspark
findspark.init()

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

# FIFO Scheduling

In [ ]:
spark = SparkSession.builder \
            .appName("FIFOScheduler") \
            .master("local[*]") \
            .config("spark.sql.autoBroadcastJoinThreshold", "10") \
            .getOrCreate()

In [ ]:
airport_df = spark.read.json("data/airlines_aiport.json")
flight_df = spark.read.json("data/airlines_flight.json")

In [ ]:
airport_df.join(flight_df, "id", "inner").show(5)

In [ ]:
spark.stop()

# Fair Scheduling

In [6]:
spark = SparkSession \
        .builder \
        .appName("FairSchedulerApp") \
        .master("local[3]") \
        .config("spark.sql.autoBroadcastJoinThreshold", "50") \
        .config("spark.scheduler.mode", "FAIR") \
        .getOrCreate()

In [7]:
import threading

In [8]:
jobs = []
output = []

In [9]:
def join_data(airport_data, flight_data):
    df1 = spark.read.json(airport_data)
    df2 = spark.read.json(flight_data)
    output.append(df1.join(df2, "id", "inner").count())

In [11]:
thread1 = threading.Thread(target=join_data, 
                           args=("data/airlines_aiport.json", "data/airlines_flight.json"))
jobs.append(thread1)

In [12]:
thread2 = threading.Thread(target=join_data, 
                           args=("data/airlines_aiport.json", "data/airlines_flight.json"))
jobs.append(thread2)

In [13]:
for job in jobs:
    job.start()

In [14]:
for job in jobs:
    job.join()

In [15]:
output

[201, 201]